In [1]:
#!pip install tsfresh

In [2]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

In [4]:
from utils import get_data
df_emg = get_data()

memory usage:  96.13 MB


In [5]:
from tsfresh.utilities.dataframe_functions import roll_time_series
# https://tsfresh.readthedocs.io/en/latest/text/forecasting.html#the-rolling-mechanism

In [6]:
df_emg['id'] = 0

In [7]:
fs = 1000
window_s = 30
stride_s = 10
df_rolled = roll_time_series(
    df_emg.reset_index().rename(columns={'index': 'timestamp'}),
    column_id='id',
    column_sort="timestamp",
    min_timeshift=window_s * fs,
    max_timeshift=window_s * fs,
    rolling_direction=stride_s * fs,
    n_jobs=0
)

Rolling: 100%|██████████| 360/360 [02:03<00:00,  2.92it/s]


In [8]:
df_rolled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10710357 entries, 0 to 10710356
Data columns (total 7 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  datetime64[ns]
 1   emg        float32       
 2   eog        float32       
 3   lso        float32       
 4   rio        float32       
 5   m1-a1      float32       
 6   id         object        
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 449.4+ MB


In [9]:
from tsfresh import extract_features

In [10]:
kind_to_fc_parameters = {
    colname: {
        x[0]: (None if len(x) == 1 else x[1])
        for x in [
            ("minimum", None),
            ("maximum",),
            ("mean",),
            ("standard_deviation",),
            ("variance",),
            ("skewness",),
            ("kurtosis",),
            ("root_mean_square",),
            ("linear_trend", [{"attr": "slope"}]),
            ("abs_energy",),
        ]
    }
    for colname in df_emg.columns
}

In [11]:
df_feat = extract_features(
    df_rolled,
    column_id="id",
    column_sort="timestamp",
    kind_to_fc_parameters=kind_to_fc_parameters,
)

Feature Extraction: 100%|██████████| 78/78 [00:03<00:00, 19.90it/s]


In [12]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 357 entries, (0, Timestamp('2021-06-29 06:36:45.067382')) to (0, Timestamp('2021-06-29 07:36:05.067382'))
Data columns (total 50 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   emg__minimum                       357 non-null    float64
 1   emg__maximum                       357 non-null    float64
 2   emg__mean                          357 non-null    float64
 3   emg__standard_deviation            357 non-null    float64
 4   emg__variance                      357 non-null    float64
 5   emg__skewness                      357 non-null    float64
 6   emg__kurtosis                      357 non-null    float64
 7   emg__root_mean_square              357 non-null    float64
 8   emg__linear_trend__attr_"slope"    357 non-null    float64
 9   emg__abs_energy                    357 non-null    float64
 10  eog__minimum                       357

In [13]:
df_feat.head(10)

emg__minimum  emg__maximum  emg__mean  \
0 2021-06-29 06:36:45.067382     -0.046875      0.062500  -0.001375   
  2021-06-29 06:36:55.067382     -1.078125      1.531250   0.035994   
  2021-06-29 06:37:05.067382     -1.312500      1.812500   0.158393   
  2021-06-29 06:37:15.067382     -1.312500      1.812500   0.282720   
  2021-06-29 06:37:25.067382     -1.312500      1.984375   0.389441   
  2021-06-29 06:37:35.067382     -0.734375      1.984375   0.525619   
  2021-06-29 06:37:45.067382     -0.734375      1.984375   0.537809   
  2021-06-29 06:37:55.067382     -0.625000      1.562500   0.496074   
  2021-06-29 06:38:05.067382     -0.562500      1.406250   0.466640   
  2021-06-29 06:38:15.067382     -0.562500      1.406250   0.425494   

                              emg__standard_deviation  emg__variance  \
0 2021-06-29 06:36:45.067382                 0.010449       0.000109   
  2021-06-29 06:36:55.067382                 0.149202       0.022261   
  2021-06-29 06:37:05.067382                 0.270045       0.072924   
  2021-06-29 06:37:15.067382                 0.256181       0.065629   
  2021-06-29 06:37:25.067382                 0.278441       0.077530   
  2021-06-29 06:37:35.067382                 0.326471       0.106583   
  2021-06-29 06:37:45.067382                 0.357847       0.128055   
  2021-06-29 06:37:55.067382                 0.339160       0.115030   
  2021-06-29 06:38:05.067382                 0.320311       0.102599   
  2021-06-29 06:38:15.067382                 0.295076       0.087070   

                              emg__skewness  emg__kurtosis  \
0 2021-06-29 06:36:45.067382       0.052670       3.248283   
  2021-06-29 06:36:55.067382       4.218570      34.075867   
  2021-06-29 06:37:05.067382       1.053494       4.567492   
  2021-06-29 06:37:15.067382       0.002594       5.115544   
  2021-06-29 06:37:25.067382       0.104877       6.625774   
  2021-06-29 06:37:35.067382      -0.015144       1.422655   
  2021-06-29 06:37:45.067382      -0.186509       0.329714   
  2021-06-29 06:37:55.067382       0.084270      -0.711299   
  2021-06-29 06:38:05.067382       0.386632      -0.640719   
  2021-06-29 06:38:15.067382       0.793497       0.202950   

                              emg__root_mean_square  \
0 2021-06-29 06:36:45.067382               0.010539   
  2021-06-29 06:36:55.067382               0.153482   
  2021-06-29 06:37:05.067382               0.313069   
  2021-06-29 06:37:15.067382               0.381522   
  2021-06-29 06:37:25.067382               0.478742   
  2021-06-29 06:37:35.067382               0.618756   
  2021-06-29 06:37:45.067382               0.645982   
  2021-06-29 06:37:55.067382               0.600931   
  2021-06-29 06:38:05.067382               0.565996   
  2021-06-29 06:38:15.067382               0.517798   

                              emg__linear_trend__attr_"slope"  \
0 2021-06-29 06:36:45.067382                    -7.435277e-08   
  2021-06-29 06:36:55.067382                     6.762936e-06   
  2021-06-29 06:37:05.067382                     1.850868e-05   
  2021-06-29 06:37:15.067382                     1.362910e-05   
  2021-06-29 06:37:25.067382                     4.245925e-06   
  2021-06-29 06:37:35.067382                     1.793347e-05   
  2021-06-29 06:37:45.067382                    -3.620793e-06   
  2021-06-29 06:37:55.067382                    -2.330793e-05   
  2021-06-29 06:38:05.067382                     1.109130e-05   
  2021-06-29 06:38:15.067382                    -2.825689e-07   

                              emg__abs_energy  ...  m1-a1__minimum  \
0 2021-06-29 06:36:45.067382         3.332520  ...        0.000000   
  2021-06-29 06:36:55.067382       706.726074  ...       -1.375000   
  2021-06-29 06:37:05.067382      2940.472412  ...       -1.406250   
  2021-06-29 06:37:15.067382      4366.921387  ...       -1.406250   
  2021-06-29 06:37:25.067382      6876.035156  ...       -1.421875   
  2021-06-29 06:37:35.067382  